# Tutorial 5 - Run experiments

In [Tutorial 4](./tutorial-4-setting-parameter-values.ipynb) we saw how to change the parameters, including the applied current. However, in some cases we might want to prescribe a given voltage, a given power or switch between different conditions to simulate experimental setups. We can use the Experiment class for these simulations.

In [1]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm
import numpy as np


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## String-based instructions

We start defining an experiment, which consists on a set of instructions on how to cycle the battery. For example, we can set the following experiment:

In [2]:
experiment = pybamm.Experiment(
    [
        ("Discharge at C/10 for 10 hours or until 3.3 V",
        "Rest for 1 hour",
        "Charge at 1 A until 4.1 V",
        "Hold at 4.1 V until 50 mA",
        "Rest for 1 hour"),
    ] * 3
)

A cycle is defined by a tuple of operating instructions. In this case, the experiment consists of a cycle of constant current C/10 discharge, a one hour rest, a constant current (1 A) constant voltage (4.1 V) and another one hour rest, all of it repeated three times (notice the * 3).

Then we can choose our model

In [3]:
model = pybamm.lithium_ion.DFN()

and create our simulation, passing our experiment using a keyword argument

In [4]:
sim = pybamm.Simulation(model, experiment=experiment)

We then solve and plot the solution

In [5]:
sim.solve()
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=38.90909528447357, step=0.3890909528447357),…

As we have seen, experiments allow us to define complex simulations using a very simple syntax. The instructions can be of the form "(Dis)charge at x A/C/W", "Rest", or "Hold at x V". The running time should be a time in seconds, minutes or hours, e.g. "10 seconds", "3 minutes" or "1 hour". The stopping conditions should be a circuit state, e.g. "1 A", "C/50" or "3 V". 

Some examples of experiment instructions are:
```python
    "Discharge at 1C for 0.5 hours",
    "Discharge at C/20 for 0.5 hours",
    "Charge at 0.5 C for 45 minutes",
    "Discharge at 1 A for 90 seconds",
    "Charge at 200mA for 45 minutes",
    "Discharge at 1 W for 0.5 hours",
    "Charge at 200 mW for 45 minutes",
    "Rest for 10 minutes",
    "Hold at 1 V for 20 seconds",
    "Charge at 1 C until 4.1V",
    "Hold at 4.1 V until 50 mA",
    "Hold at 3V until C/50",
```

Additionally, we can use the operators `+` and `*` on lists in order to combine and repeat cycles:

In [6]:
[("Discharge at 1C for 0.5 hours", "Discharge at C/20 for 0.5 hours")] * 3 + [("Charge at 0.5 C for 45 minutes",)]

[('Discharge at 1C for 0.5 hours', 'Discharge at C/20 for 0.5 hours'),
 ('Discharge at 1C for 0.5 hours', 'Discharge at C/20 for 0.5 hours'),
 ('Discharge at 1C for 0.5 hours', 'Discharge at C/20 for 0.5 hours'),
 ('Charge at 0.5 C for 45 minutes',)]

To pass additional arguments such as a period, temperature, or tags, the method `pybamm.step.string` should be used, for example:

In [7]:
pybamm.step.string("Discharge at 1C for 1 hour", period="1 minute", temperature="25oC", tags=["tag1"])

_Step(C-rate, 1.0, duration=1 hour, period=1 minute, temperature=25oC, tags=['tag1'], description=Discharge at 1C for 1 hour)

## Direct instructions

Experiments can also be specified programmatically without having to use string formatting. For example,

In [8]:
pybamm.step.current(1, duration="1 hour", termination="2.5 V")

_Step(current, 1, duration=1 hour, termination=2.5 V)

is equivalent to 

In [9]:
pybamm.step.string("Discharge at 1A for 1 hour or until 2.5V")

_Step(current, 1.0, duration=1 hour, termination=2.5V, description=Discharge at 1A for 1 hour or until 2.5V)

The available methods are `current`, `c_rate`, `voltage`, `power`, and `resistance`.

The period, temperature, and tags options are the same as for `pybamm.step.string`.

These methods can also be used for drive cycles:

In [10]:
t = np.linspace(0, 1, 60)
sin_t = 0.5 * np.sin(2 * np.pi * t)
drive_cycle_power = np.column_stack([t,sin_t])
experiment = pybamm.Experiment([pybamm.step.power(drive_cycle_power)])
sim = pybamm.Simulation(model, experiment=experiment)
sim.solve()
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

For a drive cycle, the duration is until the final time provided and the period is the smallest time step. For best results, we recommend using a constant time step size.

In this notebook we have seen how to use the Experiment class to run simulations of more complex operating conditions. In [Tutorial 6](./tutorial-6-managing-simulation-outputs.ipynb) we will see how to manage the outputs of the simulation.

## References

The relevant papers for this notebook are:

In [11]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. doi:10.1149/1.2221597.
[3] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[4] Scott G. Marquis, Valentin Sulzer, Robert Timms, Colin P. Please, and S. Jon Chapman. An asymptotic derivation of a single particle model with electrolyte. Journal of The Electrochemical Society, 166(15):A3693–A3706, 2019. doi:10.1149/